In [2]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm

In [3]:
# Paths
plate_folder = r'PlateImage'
background_train_folder = r'BackgroundImage\train'
background_testval_folder = r'BackgroundImage\testval'

output_base = r'synthetic_data'
train_output_folder = os.path.join(output_base, 'train', 'images')
train_label_folder = os.path.join(output_base, 'train', 'labels')
test_output_folder = os.path.join(output_base, 'testval', 'images')
test_label_folder = os.path.join(output_base, 'testval', 'labels')

In [4]:
# Create directories
for folder in [train_output_folder, train_label_folder, test_output_folder, test_label_folder]:
    os.makedirs(folder, exist_ok=True)

In [8]:
def overlay_plate(background, plate):
    h_bg, w_bg = background.shape[:2]
    
    # Dynamically adjust max scale to fit plate within background
    max_scale_x = w_bg / plate.shape[1]
    max_scale_y = h_bg / plate.shape[0]
    max_scale = min(max_scale_x, max_scale_y, 0.5)  # Limit to 0.5 max or lower

    # Ensure max_scale is reasonable (avoid 0 or very small)
    if max_scale < 0.1:
        max_scale = 0.1

    scale = random.uniform(0.2, max_scale)
    plate_resized = cv2.resize(plate, (0, 0), fx=scale, fy=scale)
    h_p, w_p = plate_resized.shape[:2]

    # Double-check that the resized plate fits
    if w_p >= w_bg or h_p >= h_bg:
        # Skip overlay if plate is too big; just return background with empty bbox
        return background, (0, 0, 0, 0)

    x_offset = random.randint(0, w_bg - w_p)
    y_offset = random.randint(0, h_bg - h_p)

    roi = background[y_offset:y_offset+h_p, x_offset:x_offset+w_p]
    alpha_plate = plate_resized[:, :, 3] / 255.0 if plate_resized.shape[2] == 4 else np.ones((h_p, w_p))
    for c in range(3):
        roi[:, :, c] = (1 - alpha_plate) * roi[:, :, c] + alpha_plate * plate_resized[:, :, c]
    background[y_offset:y_offset+h_p, x_offset:x_offset+w_p] = roi

    x_center = (x_offset + w_p/2) / w_bg
    y_center = (y_offset + h_p/2) / h_bg
    width = w_p / w_bg
    height = h_p / h_bg

    return background, (x_center, y_center, width, height)


In [9]:
# Load lists of images
plate_files = [os.path.join(plate_folder, f) for f in os.listdir(plate_folder) if f.lower().endswith(('.png', '.jpg'))]
train_bg_files = [os.path.join(background_train_folder, f) for f in os.listdir(background_train_folder) if f.lower().endswith(('.png', '.jpg'))]
testval_bg_files = [os.path.join(background_testval_folder, f) for f in os.listdir(background_testval_folder) if f.lower().endswith(('.png', '.jpg'))]

# Updated generation parameters
num_train_images = len(plate_files) * 3  # ~6500 images
num_testval_images = len(plate_files)    # ~2167 images

In [10]:
# Generate training images
print("Generating training set...")
for i in tqdm(range(num_train_images)):
    bg_path = random.choice(train_bg_files)
    plate_path = random.choice(plate_files)

    bg = cv2.imread(bg_path)
    plate = cv2.imread(plate_path, cv2.IMREAD_UNCHANGED)
    bg = cv2.resize(bg, (640, 480))

    result_img, bbox = overlay_plate(bg, plate)

    img_name = f"train_{i:05d}.jpg"
    cv2.imwrite(os.path.join(train_output_folder, img_name), result_img)
    label_path = os.path.join(train_label_folder, f"train_{i:05d}.txt")
    with open(label_path, 'w') as f:
        f.write(f"0 {' '.join(map(str, bbox))}\n")

# Generate test/validation images
print("Generating test/validation set...")
for i in tqdm(range(num_testval_images)):
    bg_path = random.choice(testval_bg_files)
    plate_path = random.choice(plate_files)

    bg = cv2.imread(bg_path)
    plate = cv2.imread(plate_path, cv2.IMREAD_UNCHANGED)
    bg = cv2.resize(bg, (640, 480))

    result_img, bbox = overlay_plate(bg, plate)

    img_name = f"testval_{i:05d}.jpg"
    cv2.imwrite(os.path.join(test_output_folder, img_name), result_img)
    label_path = os.path.join(test_label_folder, f"testval_{i:05d}.txt")
    with open(label_path, 'w') as f:
        f.write(f"0 {' '.join(map(str, bbox))}\n")

Generating training set...


100%|██████████████████████████████████████████████████████████████████████████████| 6501/6501 [01:37<00:00, 66.99it/s]


Generating test/validation set...


100%|██████████████████████████████████████████████████████████████████████████████| 2167/2167 [00:42<00:00, 51.40it/s]
